In [1]:
# Import TensorFlow / Keras
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Embedding, Reshape, Concatenate, Flatten, Dropout
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPool2D, ReLU, LeakyReLU
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam

# Data manipulation
import numpy as np
print('numpy: %s' % np.__version__)

# Visualization
import matplotlib 
import matplotlib.pyplot as plt
print('matplotlib: %s' % matplotlib.__version__)
#import graphviz
#print('graphviz: %s' % graphviz.__version__)

# Other utilities
import sys
import os

# Assign main directory to a variable
main_dir = os.path.dirname(sys.path[0])
#print(main_dir)

2023-11-27 18:01:25.062300: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


numpy: 1.24.3
matplotlib: 3.7.1


In [13]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
import matplotlib.pyplot as plt

# Set the path to your album_covers folder
folder_path = "./album_covers"

# Initialize empty lists to store data
images = []
labels = []

# Iterate through files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".jpeg"):
        # Extract genre and label from the filename
        genre, label = filename.split("_")
        label = int(label.split(".")[0])  # Remove ".jpeg" extension and convert to integer
       
        # Read the image and resize it to 256x256 pixels
        image = cv2.imread(os.path.join(folder_path, filename), cv2.COLOR_RGB2BGR)
        image = cv2.resize(image, (256, 256))
        
        # Normalize the image to [0, 1] range
        image = image / 255.0

        # Append the image and label to the lists
        images.append(image)
        labels.append(label)

# Convert lists to numpy arrays
x_train = np.array(images)
y_train = np.array(labels)

# Split the data into training and testing sets (you can adjust the test_size as needed)
x_train, _, y_train, _ = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Print shapes
print("Shape of x_train: ", x_train.shape)
print("Shape of y_train: ", y_train.shape)

# Display images of the first 10 album covers in the training set and their true labels
fig, axs = plt.subplots(2, 5, sharey=False, tight_layout=True, figsize=(12,6))
for i in range(2):
    for j in range(5):
        axs[i, j].imshow(x_train[j + i * 5])
        axs[i, j].set_title(y_train[j + i * 5])
        axs[i, j].axis('off')
plt.show()

# Reshape and scale the data as required by the model
x_train = x_train.reshape(x_train.shape[0], 256, 256, 1)
x_train = (x_train - 0.5) / 0.5  # Normalize the images to [-1, 1]
print("Shape of the scaled array: ", x_train.shape)

error: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - cvtColor() missing required argument 'code' (pos 2)
>  - cvtColor() missing required argument 'code' (pos 2)
